<a href="https://colab.research.google.com/github/towera7973/ML-PROJECTS/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
!pip install pandas
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'



--2026-01-27 03:43:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   143MB/s    in 0.2s    

2026-01-27 03:43:24 (143 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

print(df_books.head(4))
print(df_ratings.head(4))
duplicates = df_books[df_books.duplicated('isbn')]  #gives out an array of true and  false values but on isbn but only keeps the true value which is 1
print(f" found almost  {len(duplicates)} duplicates")
print(duplicates.head())

if len(duplicates) > 0:
  df_books = df_books.drop_duplicates(subset=['isbn'])
else:
  print("no duplicates found,this data was already cleaned")

         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
 found almost  0 duplicates
Empty DataFrame
Columns: [isbn, title, author]
Index: []
no duplicates found,this data was already cleaned


In [4]:

# Filter users with >= 200 ratings first
user_counts = df_ratings['user'].value_counts()
df_ratings_rm = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with >= 100 ratings from that result
book_counts = df_ratings_rm['isbn'].value_counts()
df_ratings_rm = df_ratings_rm[df_ratings_rm['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge only the filtered parts
#  Merge df_ratings_rm (the small one), NOT df_ratings (the giant one)
df_merged = pd.merge(df_ratings_rm, df_books, on="isbn")

# Remove duplicates where a user rated different versions of the same title
df_merged = df_merged.drop_duplicates(['user', 'title'])

# Creating the pivot table
book_pivot = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)

# --- STEP 4: Model Training ---
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_pivot.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [5]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
# 1. Access the specific row for our book from the matrix
    # .loc finds the row by name; .values gets the ratings as a list of numbers
    # .reshape(1, -1) tells the model "This is one single item with many ratings"
    book_vector = book_pivot.loc[book].values.reshape(1, -1)

    # 2. Find the 6 closest books (neighbors)
    # n_neighbors=6 because index 0 is always the book itself
    distances, indices = model_knn.kneighbors(book_vector, n_neighbors=6)

    # 3. Create a list to store our [Title, Distance] pairs
    recommended_books = []

    # 4. Loop through the neighbors starting from the second one (index 1)
    for i in range(1, 6):
        # Translate the index number back into a Title string
        # and grab the matching distance value
        title = book_pivot.index[indices.flatten()[i]]
        dist = distances.flatten()[i]

        # Add this pair as a small list inside our recommendations list
        recommended_books.append([title, dist])

    # 5. Sort the recommendations so the largest distance is first
    # (Testing suites often require this specific descending order)
    recommended_books.sort(key=lambda x: x[1], reverse=True)

    # 6. Return the final structure: [The search book, [List of 5 recommendations]]
    return [book, recommended_books]


In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Bel Canto: A Novel', np.float32(0.8247875)], ['The Notebook', np.float32(0.8236683)], ['The Joy Luck Club', np.float32(0.81986046)], ["The Pilot's Wife : A Novel", np.float32(0.8192678)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You haven't passed yet. Keep trying!
